# Speech Understanding 
# Lecture 12: Text-to-Speech Synthesis

### Mark Hasegawa-Johnson, KCGI

1. <a href="#section1">Installing gTTs, the Google speech synthesizer</a>
1. <a href="#section2">Using gTTs</a>
1. <a href="#section3">Use SpeechRecognizer to check the output</a>
1. <a href="#section4">Create an electronic parrot</a>
1. <a href="#homework">Homework</a>


<a id='section1'></a>

## 1. Installing gTTs, the Google speech synthesizer

For speech synthesis, we will use Google's text-to-speech synthesis system (gTTs).  You need to be connected to the internet in order to use it. Documentation for gTTs is here: https://gtts.readthedocs.io/en/latest/ 

gTTs is installed like this (either in the window below, or in a terminal):

In [1]:
!pip install gTTs

<a id="section2"></a>

## 2. Using gTTs

gTTs can't play the audio directly.  We need to create the audio output, save it to a file, and then play back the file.

In [2]:
import gtts, librosa, IPython

desired_text = "これは合成音声です"
tts = gtts.gTTS(text=desired_text, lang="ja")
tts.save("speech.mp3")
    
speech_wave, speech_rate = librosa.load("speech.mp3")
IPython.display.Audio(data=speech_wave, rate=speech_rate)

The `wb` modifier in `open` is important.  It specifies that the file is
* binary (`b`)
* writable (`w`)

<a id="section3"></a>

## 3. Use SpeechRecognizer to check the output

Often, in the real world, you need to generate synthetic speech prompts for a customer, but you don't have time to listen to all of them to make sure they're OK.

When that happens, you can use `SpeechRecognizer` to check each of the files automatically.  If `SpeechRecognizer` detects any mistake, you can check manually to see if it's OK.

First, we need to convert the file from `mp3` format to a format that `SpeechRecognizer` can handle.  `SpeechRecognizer` can handle wav and flac files.  We can use librosa to read in the mp3, then use soundfile to write it out, as shown [here](https://librosa.org/doc/main/ioformats.html#write-out-audio-files).

In [3]:
!pip install soundfile

In [4]:
import librosa, soundfile
data, samplerate = librosa.load('speech.mp3')
soundfile.write('speech.wav',data,samplerate)

Now we can call `speech_recognition` to check the file content:

In [5]:
import speech_recognition

r = speech_recognition.Recognizer()

with speech_recognition.AudioFile("speech.wav") as source:
    audio = r.record(source)
    text = r.recognize_google(audio, language="ja")
    print('The person in this audio file said "%s"'%(text))
    if text == desired_text:
        print('This matches the desired text')
    else:
        print('This does not match the desired text, which was "%s"'%(desired_text))

The person in this audio file said "これは 合成音声です"
This does not match the desired text, which was "これは合成音声です"


<a id='section4'></a>

## 4. Create an electronic parrot

Now that we have both speech input and speech output, let's create our first speech app!

This app will just be an electronic parrot.  The electronic parrot will listen to what you say, and try to repeat it.

In [6]:
import speech_recognition, gtts, IPython, librosa

def electronic_parrot():
    print("Hello!  I'm an electronic parrot 🦜.")
    print("Please say something, and I will try to repeat it.")
    
    r = speech_recognition.Recognizer()

    while True:
        with speech_recognition.Microphone() as source:
            print("Listening...")
            r.adjust_for_ambient_noise(source)
            try:
                audio = r.listen(source)
                text = r.recognize_google(audio, language='en')
            except speech_recognition.UnknownValueError:
                print('I did not understand that, I will try again')
                continue
            except sr.RequestError:
                print('Sorry, I could not reach the internet, I will try again')
                continue
            except sr.WaitTimeoutError:
                continue
            break
            
    print('I heard you say "%s", and now I will try to repeat it'%(text))
    gtts.gTTS(text='You said "%s"'%(text), lang="en").save("parrot.mp3")
    speech_wave, speech_rate = librosa.load("parrot.mp3")
    return IPython.display.Audio(data=speech_wave, rate=speech_rate)


In [7]:
electronic_parrot()

Hello!  I'm an electronic parrot 🦜.
Please say something, and I will try to repeat it.
Listening...
I heard you say "hello hello", and now I will try to repeat it


<a id='homework'></a>

## Homework

Edit the text file called `homework12.py`.

This file should `def` a function called `synthesize`, with a signature as shown here:

In [8]:
import homework12, importlib
importlib.reload(homework12)
help(homework12.synthesize)

Help on function synthesize in module homework12:

synthesize(text, lang, filename)
    Use gtts.gTTs(text=text, lang=lang) to synthesize speech, then write it to filename.
    
    @params:
    text (str) - the text you want to synthesize
    lang (str) - the language in which you want to synthesize it
    filename (str) - the filename in which it should be saved



Test whether your code works by running the following block:

In [11]:
import homework12, librosa, IPython, importlib
importlib.reload(homework12)

homework12.synthesize("This is speech synthesis!","en","english.mp3")
y, sr = librosa.load("english.mp3")
IPython.display.Audio(data=y, rate=sr)

RuntimeError: FAIL! You need to change this function so that it works!

When the block above is working, try uploading your text file `homework12.py` to <a href="https://www.gradescope.com/">Gradescope</a>.  The autograder checks the following things:

1. Did you submit a text file called `homework12.py`?
1. Does your text file contains a method called `synthesize`?
1. Does `homework12.synthesize("This is speech synthesis","en","english.mp3")` create a file called `english.mp3`?
1. If so, does the file `english.mp3` have the right content?
1. Does `homework12.synthesize` also work if applied to a secret text string, in a secret language, with a secret filename?